In [41]:
import numpy as np
from copy import deepcopy

size = Lx, Ly = (600., 800.)
E = 2890.
nu = 0.2
t = 50.

In [42]:
G = E/2/(1+nu)
D = np.array([[1, nu, 0], [nu, 1, 0], [0., 0, (1-nu)/2]]) * t**3 * (E / (1-nu**2)) / 12
S = np.array([[G, 0], [0, G]]) * t * 5 / 6

In [43]:
loads = {
    'LG1' : {
        'LC1' : {
            'type' : 'rect',
            'points' : [[0, 0], [Lx, Ly]],
            'value' : [0, 0, -0.01],
                },
        'LC2' : {
            'type' : 'rect',
            'region' : [0.2*Lx, 0.5*Ly, 0.2*Lx, 0.3*Ly],
            'value' : [0, 0, -0.1],
                }
            },
    'LG2' : {
        'LC3' : {
            'type' : 'point',
            'point' : [Lx/3, Ly/2],
            'value' : [0, 0, -10],
                },
        'LC4' : {
            'type' : 'point',
            'point' : [2*Lx/3, Ly/2],
            'value' : [0, 0, 10],
                }
            },
    'dummy1' : 10
        }


In [44]:
from dewloosh.geom import grid, PolyData
from dewloosh.geom.tri.trimesh import triangulate
from dewloosh.geom.topo.tr import Q4_to_T3
from dewloosh.geom.tri.triplot import triplot
import matplotlib.pyplot as plt
from matplotlib import gridspec
plt.style.use('default')

shape = nx, ny = (30, 40)
gridparams = {
    'size' : size,
    'shape' : shape,
    'origo' : (0, 0),
    'start' : 0,
    'eshape' : 'Q4'
    }
coords_, topo = grid(**gridparams)
coords = np.zeros((coords_.shape[0], 3))
coords[:, :2] = coords_[:, :]
del coords_
coords, triangles = Q4_to_T3(coords, topo)

triobj = triangulate(points=coords[:, :2], triangles=triangles)[-1]
Mesh = PolyData(coords=coords, topo=triangles)
centers = Mesh.centers()

In [45]:
from dewloosh.solid.navier import NavierProblem

Mindlin = NavierProblem(size, (50, 50), D=D, S=S, model='mindlin')
LoadsM = Mindlin.add_loads_from_dict(deepcopy(loads))
Mindlin.solve()
Mindlin.postproc(centers[:, :2], cleanup=False)
res2dM = LoadsM['LG1', 'LC2'].res2d

Kirchhoff = NavierProblem(size, (50, 50), D=D, S=S, model='kirchhoff')
LoadsK = Kirchhoff.add_loads_from_dict(deepcopy(loads))
Kirchhoff.solve()
Kirchhoff.postproc(centers[:, :2], cleanup=False)
res2dK = LoadsK['LG1', 'LC2'].res2d

In [47]:
UZ, ROTX, ROTY, CX, CY, CXY, EXZ, EYZ, MX, MY, MXY, QX, QY = list(range(13))
labels = {UZ : 'UZ', ROTX : 'ROTX', ROTY : 'ROTY', CX : 'CX', 
          CY : 'CY', CXY : 'CXY', EXZ : 'EXZ', EYZ : 'EYZ', 
          MX : 'MX', MY : 'MY', MXY : 'MXY', QX : 'QX', QY : 'QY'}

In [48]:
%matplotlib qt

In [49]:
res2d = 100 * (res2dK - res2dM) / res2dM

fig = plt.figure(figsize=(8, 3))  # in inches
fig.patch.set_facecolor('white')
cmap = 'jet'
gs = gridspec.GridSpec(1, 3)
for i, key in enumerate([UZ, ROTX, ROTY]):
    ax = fig.add_subplot(gs[i])
    triplot(triobj, ax=ax, fig=fig, title=labels[key],
            data=res2d[key, :], cmap=cmap, axis='off')
fig.tight_layout()

fig = plt.figure(figsize=(12, 3))  # in inches
fig.patch.set_facecolor('white')
cmap = 'seismic'
gs = gridspec.GridSpec(1, 5)
for i, key in enumerate([MX, MY, MXY, QX, QY]):
    ax = fig.add_subplot(gs[i])
    triplot(triobj, ax=ax, fig=fig, title=labels[key],
            data=res2d[key, :], cmap=cmap, axis='off')
fig.tight_layout()

In [15]:
from dewloosh.geom import mesh1d_uniform
from dewloosh.geom.utils import cell_coords_bulk, pcoords_to_coords
from dewloosh.solid.navier.plot import plot_path

In [16]:
coords = np.array([[0., 0, 0.], [0, Ly, 0]], dtype=float)
topo = np.array([[0, 1]], dtype=np.int32)
coords, topo = mesh1d_uniform(coords, topo, 2, N=50)

ecoords = cell_coords_bulk(coords, topo[0])
pcoords = np.array([0.0, 1.0])
datacoords = pcoords_to_coords(pcoords, ecoords)

Kirchhoff.postproc(datacoords[:, :2], cleanup=False)
dataK = LoadsK['LG1', 'LC1'].res2d

Mindlin.postproc(datacoords[:, :2], cleanup=False)
dataM = LoadsM['LG1', 'LC1'].res2d

fig = plt.figure(figsize=(20, 4))  # in inches
fig.patch.set_facecolor('white')
gs = gridspec.GridSpec(1, 5, width_ratios=np.full(5, 1))
for i, key in enumerate([MX, MY, MXY, QX, QY]):
    ax = fig.add_subplot(gs[i])
    plot_path(ax, dataK[key], datacoords, normalize=False, 
              markersize=10, title=labels[key])
    plot_path(ax, dataM[key], datacoords, normalize=False, 
              markersize=5, markercolor='b', title=labels[key])    
fig.suptitle('Load Case 2') 

Text(0.5, 0.98, 'Load Case 2')

In [17]:
coords = np.array([[0., 0., 0.], [Lx, 0, 0],
                   [Lx, Ly, 0], [0, Ly, 0]], dtype=float)
topo = np.array([[0, 1], [1, 2], [2, 3], [3, 0]], dtype=np.int32)
coords, topo = mesh1d_uniform(coords, topo, 2, N=50)
topo = np.vstack(list(topo[i] for i in range(len(topo))))

ecoords = cell_coords_bulk(coords, topo)
pcoords = np.array([0.0, 1.0])
datacoords = pcoords_to_coords(pcoords, ecoords)

Kirchhoff.postproc(datacoords[:, :2], cleanup=False)
data = LoadsK['LG1', 'LC1'].res2d

fig = plt.figure(figsize=(8, 3))  # in inches
fig.patch.set_facecolor('white')
gs = gridspec.GridSpec(1, 2, width_ratios=np.full(2, 1))
for i, key in enumerate([QX, QY]):
    plot_path(fig.add_subplot(gs[i]), data[key], datacoords, 
              normalize=False, markersize=10, title=labels[key])    
fig.suptitle('Load Case 2') 

Text(0.5, 0.98, 'Load Case 2')

In [18]:
data[QX].max()

2.385096082125504

In [19]:
coords = np.array([[0., 0., 0.], [Lx, 0, 0],
                   [Lx, Ly, 0], [0, Ly, 0]], dtype=float)
topo = np.array([[0, 1], [1, 2], [2, 3], [3, 0]], dtype=np.int32)
coords, topo = mesh1d_uniform(coords, topo, 2, N=50)
topo = np.vstack(list(topo[i] for i in range(len(topo))))

ecoords = cell_coords_bulk(coords, topo)
pcoords = np.array([0.0, 1.0])
datacoords = pcoords_to_coords(pcoords, ecoords)

Mindlin.postproc(datacoords[:, :2], cleanup=False)
data = LoadsM['LG1', 'LC1'].res2d

fig = plt.figure(figsize=(8, 3))  # in inches
fig.patch.set_facecolor('white')
gs = gridspec.GridSpec(1, 2, width_ratios=np.full(2, 1))
for i, key in enumerate([QX, QY]):
    plot_path(fig.add_subplot(gs[i]), data[key], datacoords, 
              normalize=False, markersize=10, title=labels[key])    
fig.suptitle('Load Case 2') 

Text(0.5, 0.98, 'Load Case 2')

In [20]:
data[QX].max()

2.385096082125496

In [21]:
import sympy as sy
from sympy import symbols, Matrix, pycode, simplify

In [22]:
P11, P12, P13, P21, P22, P23, P31, P32, P33 = \
    symbols('P_{11} P_{12} P_{13} P_{21} P_{22} P_{23} P_{31} \
            P_{32} P_{33}', real = True)
Pij = [[P11, P12, P13], [P21, P22, P23], [P31, P32, P33]]
P = sy.Matrix(Pij)
detP = P.det()
adjP = P.adjugate()
invP = adjP / detP
subs = {P11 : 'P11', P12 : 'P12', P13 : 'P13', P21:'P21', 
        P22 : 'P22', P23 : 'P23', P31 : 'P31', P32 : 'P32', 
        P33 : 'P33'}


In [23]:
pycode(simplify(adjP.subs(subs)))

'ImmutableDenseMatrix([[P22*P33 - P23*P32, -P12*P33 + P13*P32, P12*P23 - P13*P22], [-P21*P33 + P23*P31, P11*P33 - P13*P31, -P11*P23 + P13*P21], [P21*P32 - P22*P31, -P11*P32 + P12*P31, P11*P22 - P12*P21]])'

In [24]:
pycode(simplify(detP.subs(subs)))

'P11*P22*P33 - P11*P23*P32 - P12*P21*P33 + P12*P23*P31 + P13*P21*P32 - P13*P22*P31'